# Homework 5 : WeatherPy using Openweathermap API and Citipy

In [51]:
import openweathermapy.core as ow
from citipy import citipy as cp
import pandas as pd
import numpy as np
import random
import requests as req
import json
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [42]:
# Save config information.
api_key = "bfe8624363d9fae0c73d21055be1bfe8"
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = url + "appid=" + api_key + "&units=" + units + "&q="            

In [32]:
# Create dictionary of random coordinates
lat = []
lon = []
random_coords = {'Latitude': lat,'Longitude': lon}
for i in range(0,1000): 
    ran_lat = random.uniform(-90, 90)
    lat.append(ran_lat)  
    ran_lon = random.uniform(-180,180)
    lon.append(ran_lon)

In [33]:
# Check number of coordinates
len(random_coords['Latitude'])

1000

In [34]:
#Create dataframe for coordinates
df = pd.DataFrame(random_coords)
df.head()

,Latitude,Longitude
0,-89.273076,13.658701
1,-59.258141,172.992265
2,0.121894,103.662916
3,86.812021,-128.615188
4,60.292383,56.191607


In [35]:
# Create additional colums for City, Temperature (F), Humidity (%), Cloudiness (%), Wind Speed (mph)
df['City'] = ""
df['Country'] = ""
df['Temperature (F)'] = ""
df['Humidity (%)'] = ""
df['Cloudiness (%)'] = ""
df['Wind Speed (mph)'] = ""
df.head(2)

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,-89.273076,13.658701,,,,,,
1,-59.258141,172.992265,,,,,,


In [36]:
# Loop through coordinates in df with Citipy to return cities
row_count = 1
for index, row in df.iterrows():
   
    city = cp.nearest_city((row["Latitude"]), (row["Longitude"]))
    return_city = city.city_name
    return_country = city.country_code
    
    df.set_value(index, "City", return_city)
    df.set_value(index, "Country", return_country) 
     
    # Print log to ensure loop is working correctly
    print("Now retrieving city # " + str(row_count) + " " + return_city + ", " + return_country)
    row_count += 1 
    
df

Now retrieving city # 1 bredasdorp, za
Now retrieving city # 2 kaitangata, nz
Now retrieving city # 3 mendahara, id
Now retrieving city # 4 tuktoyaktuk, ca
Now retrieving city # 5 cherdyn, ru
Now retrieving city # 6 barentsburg, sj
Now retrieving city # 7 nome, us
Now retrieving city # 8 punta arenas, cl
Now retrieving city # 9 awjilah, ly
Now retrieving city # 10 katsuura, jp
Now retrieving city # 11 chuy, uy
Now retrieving city # 12 belushya guba, ru
Now retrieving city # 13 barrow, us
Now retrieving city # 14 hihifo, to
Now retrieving city # 15 atuona, pf
Now retrieving city # 16 formoso do araguaia, br
Now retrieving city # 17 tilichiki, ru
Now retrieving city # 18 cape town, za
Now retrieving city # 19 vestmannaeyjar, is
Now retrieving city # 20 samarai, pg
Now retrieving city # 21 galesong, id
Now retrieving city # 22 pochutla, mx
Now retrieving city # 23 trairi, br
Now retrieving city # 24 satitoa, ws
Now retrieving city # 25 cape town, za
Now retrieving city # 26 provideniya, r

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,-89.273076,13.658701,bredasdorp,za,,,,
1,-59.258141,172.992265,kaitangata,nz,,,,
2,0.121894,103.662916,mendahara,id,,,,
3,86.812021,-128.615188,tuktoyaktuk,ca,,,,
4,60.292383,56.191607,cherdyn,ru,,,,
5,77.749813,12.740307,barentsburg,sj,,,,
6,65.833225,-165.369143,nome,us,,,,
7,-61.095051,-92.055988,punta arenas,cl,,,,
8,25.015303,19.782281,awjilah,ly,,,,
9,27.032539,145.555183,katsuura,jp,,,,


In [46]:
# Loop through all cities using OpenWeatherMap to return weather info
row_count = 1
for index, row in df.iterrows():
    
    city = row["City"]
    target_url = (query_url + city)
    
    try:
        weather_data = [req.get(target_url).json()]
        return_temp = weather_data[0]["main"]["temp"]
        return_humid = weather_data[0]["main"]["humidity"]
        return_cloud = weather_data[0]["clouds"]["all"]
        return_wind = weather_data[0]["wind"]["speed"]
    
        df.set_value(index, "Temperature (F)", return______)
        df.set_value(index, "Humidity (%)", return_______) 
        df.set_value(index, "Cloudiness (%)", return______)
        df.set_value(index, "Wind Speed (mph)", return_______)  
    
        # Print log to ensure loop is working correctly
        print("Now retrieving city # " + str(row_count) + " " + row["City"].title() + ": " + target_url)
        row_count += 1 
        
    except:
        continue
    
#df

Now retrieving city # 1 Bredasdorp: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=bredasdorp
Now retrieving city # 2 Kaitangata: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=kaitangata
Now retrieving city # 3 Mendahara: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=mendahara
Now retrieving city # 4 Tuktoyaktuk: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=tuktoyaktuk
Now retrieving city # 5 Cherdyn: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=cherdyn
Now retrieving city # 6 Barentsburg: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=barentsburg
Now retrieving city # 7 Nome: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d2

Now retrieving city # 904 Chertkovo: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=chertkovo
Now retrieving city # 905 Puerto Ayora: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=puerto ayora
Now retrieving city # 906 New Norfolk: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=new norfolk
Now retrieving city # 907 Airai: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=airai
Now retrieving city # 908 Zabid: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=zabid
Now retrieving city # 909 Port Alfred: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=port alfred
Now retrieving city # 910 Nantucket: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9

In [52]:
weather_data = [req.get(target_url).json()]
weather_data

[{'base': 'stations',
  'clouds': {'all': 64},
  'cod': 200,
  'coord': {'lat': 47.75, 'lon': 38.03},
  'dt': 1503376152,
  'id': 692829,
  'main': {'grnd_level': 998.76,
   'humidity': 32,
   'pressure': 998.76,
   'sea_level': 1021.49,
   'temp': 76.01,
   'temp_max': 76.01,
   'temp_min': 76.01},
  'name': 'Starobesheve',
  'sys': {'country': 'UA',
   'message': 0.0039,
   'sunrise': 1503369175,
   'sunset': 1503419252},
  'weather': [{'description': 'broken clouds',
    'icon': '04d',
    'id': 803,
    'main': 'Clouds'}],
  'wind': {'deg': 111, 'speed': 13.67}}]

In [56]:
weather_data[0]["main"]["temp"]

76.01

In [57]:
weather_data[0]["main"]["humidity"]

32

In [58]:
weather_data[0]["clouds"]["all"]

64

In [59]:
weather_data[0]["wind"]["speed"]

13.67